## Get users associated with a given repo

In [2]:
from github3 import login
import helper as h
password = h.get_gh_credentials()
gh = login('bps10', password=password)

In [7]:
repo = gh.repository('numpy', 'numpy')

In [14]:
#contributors = repo.contributors()

for contributor in repo.contributors():
    print([contributor.login, contributor.contributions])

['charris', 4090]
['teoliphant', 2065]
['cournape', 1525]
['pearu', 1061]
['eric-wieser', 866]
['pv', 762]
['mwiebe', 759]
['juliantaylor', 745]
['rgommers', 586]
['stefanv', 449]
['seberg', 443]
['mattip', 385]
['njsmith', 339]
['jaimefrio', 255]
['eric-jones', 248]
['ahaldane', 246]
['mhvk', 227]
['pierregm', 199]
['rkern', 190]
['shoyer', 167]
['jarrodmillman', 136]
['jayvius', 127]
['tylerjereddy', 94]
['matthew-brett', 93]
['abalkin', 90]
['larsmans', 64]
['87', 56]
['alexbrc', 52]
['certik', 52]
['cgohlke', 52]
['endolith', 49]
['jakirkham', 49]
['gfyoung', 47]
['MSeifert04', 46]
['mdboom', 43]
['WarrenWeckesser', 37]
['madphysicist', 34]
['huard', 32]
['dwf', 30]
['m-d-w', 30]
['timleslie', 29]
['ewmoore', 26]
['chanley', 25]
['chatcannon', 24]
['nouiz', 24]
['j-towns', 24]
['arinkverma', 23]
['cowlicks', 23]
['fperez', 22]
['hmaarrfk', 22]
['anntzer', 21]
['ahojnnes', 21]
['jsalvatier', 18]
['jjhelmus', 18]
['pitrou', 18]
['astrofrog', 16]
['aarchiba', 15]
['jonathanunderwood',

In [12]:
contributor.as_dict()

{'login': 'denisalevi',
 'id': 11474000,
 'node_id': 'MDQ6VXNlcjExNDc0MDAw',
 'avatar_url': 'https://avatars2.githubusercontent.com/u/11474000?v=4',
 'gravatar_id': '',
 'url': 'https://api.github.com/users/denisalevi',
 'html_url': 'https://github.com/denisalevi',
 'followers_url': 'https://api.github.com/users/denisalevi/followers',
 'following_url': 'https://api.github.com/users/denisalevi/following{/other_user}',
 'gists_url': 'https://api.github.com/users/denisalevi/gists{/gist_id}',
 'starred_url': 'https://api.github.com/users/denisalevi/starred{/owner}{/repo}',
 'subscriptions_url': 'https://api.github.com/users/denisalevi/subscriptions',
 'organizations_url': 'https://api.github.com/users/denisalevi/orgs',
 'repos_url': 'https://api.github.com/users/denisalevi/repos',
 'events_url': 'https://api.github.com/users/denisalevi/events{/privacy}',
 'received_events_url': 'https://api.github.com/users/denisalevi/received_events',
 'type': 'User',
 'site_admin': False,
 'contributions

In [56]:
user = gh.user('bps10')

In [60]:
user_events = user.events(number=-1, public=True)

In [62]:
for event in user_events:
    print(event.type)

PushEvent
PushEvent
CreateEvent
CreateEvent
PushEvent
PushEvent
PushEvent
PushEvent
PublicEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
CreateEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
MemberEvent
PushEvent
PushEvent
PushEvent
MemberEvent
PushEvent


In [37]:
event.created_at

datetime.datetime(2018, 11, 2, 12, 30, 3, tzinfo=tzutc())

In [38]:
user_events.next().created_at

datetime.datetime(2019, 1, 25, 2, 0, 32, tzinfo=tzutc())

In [63]:
event.ratelimit_remaining

4952

In [70]:
user_events = user.events(number=500, public=True)
for event in user_events:
    print(event.type)

PushEvent
PushEvent
CreateEvent
CreateEvent
PushEvent
PushEvent
PushEvent
PushEvent
PublicEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
CreateEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
MemberEvent
PushEvent
PushEvent
PushEvent
MemberEvent
PushEvent
